In [93]:
from matplotlib import pyplot as plt
from urllib import request
import pandas as pd
import numpy as np
import os
import requests, json
import cv2

In [ ]:
BASE_URI = 'https://api.cognitive.microsoft.com/bing/v7.0/images/visualsearch'
HEADERS = {'Ocp-Apim-Subscription-Key': 'f62f5db4efa6496995702b547caa8386'}

In [24]:
def auxRectangles(top_left, bottom_right, shape):
    
    new_top_left = int(shape[1]*top_left['x']),int(shape[0]*top_left['y'])
    
    new_bottom_right = int(shape[1]*bottom_right['x']),int(shape[0]*bottom_right['y'])
    
    return(new_top_left, new_bottom_right)

In [106]:
images=os.listdir('C:/Users/Igor.Liberto/Desktop/produtos_clean')
for image in images:
    imagePath = 'C:/Users/Igor.Liberto/Desktop/produtos_clean/' + image
    sku = image.split('.')[0]
    file = {'image' : ('myfile', open(imagePath, 'rb'))}
    img = cv2.imread(imagePath)
    response = requests.post(BASE_URI, headers=HEADERS, files=file)
    response.raise_for_status()
    resultados={}

    for element in response.json()['tags']:
        if np.shape(element['actions'])[0] == 2:
            if element['actions'][0]['actionType'] == 'VisualSearch':
                top_left, bottom_right = auxRectangles(element['boundingBox']['queryRectangle']['topLeft'],\
                                               element['boundingBox']['queryRectangle']['bottomRight'],\
                                               img.shape)
                resultados[element['displayName']]={'topLeft':top_left,'bottomRight':bottom_right}
            
                
    for key in resultados.keys():
        tl = resultados[key]['topLeft']
        br = resultados[key]['bottomRight']
        x,y=tl
        w = br[0]-tl[0]
        h = br[1]-tl[1]
        crop_img = img[y:y+h, x:x+w]
        cv2.imwrite('C:/Users/Igor.Liberto/Desktop/produtos_cropped/'+sku+'_'+key+'.png',crop_img)
        